In [178]:
import re
import copy
import collections
import csv
import json

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords

# Data Preprocessing:

In [179]:
df_2019_emp = pd.read_excel('AY 2019-BBE-EMPFreeResponse.xlsx',sheet_name=2, skiprows=1, index_col=0)
df_2020_emp = pd.read_excel('AY 2020-BBE-EMP Free Response.xlsx',sheet_name=2, skiprows=1, index_col=0) 
df_2021_emp = pd.read_excel('AY 2021 BBE EFR Free Response.xlsx',sheet_name=2, skiprows=1, index_col=0) 

In [180]:
df_2019_emp['year'] = "2019"
df_2020_emp['year'] = "2020"
df_2021_emp['year'] = "2021"

In [181]:
df = pd.concat([df_2019_emp,df_2020_emp,df_2021_emp])
df = pd.DataFrame(df)

In [182]:
df = df.rename(columns={"What changes, if any, would you suggest for the academic preparation of this student?": "suggestions"})
df.head(2)

,Work Terms,College,Major,Concentration,Class BOT,Citizenship Status,Co-op #,suggestions,year,Terms,Coll,Conc 1,Citizenship,Coop #
Responder ID,,,,,,,,,,,,,,
12,201915-201925,B,FIN,5COP,JR,Domestic,Third,Please note that this is not directly related ...,2019,NaN,NaN,NaN,NaN,NaN
27,201935-201945,B,BAE,5COP,SR,Domestic,Third,N/A - Perfect fit,2019,NaN,NaN,NaN,NaN,NaN


In [183]:
df['Clean_Text'] = df['suggestions'].str.lower()
df.head(2)

,Work Terms,College,Major,Concentration,Class BOT,Citizenship Status,Co-op #,suggestions,year,Terms,Coll,Conc 1,Citizenship,Coop #,Clean_Text
Responder ID,,,,,,,,,,,,,,,
12,201915-201925,B,FIN,5COP,JR,Domestic,Third,Please note that this is not directly related ...,2019,NaN,NaN,NaN,NaN,NaN,please note that this is not directly related ...
27,201935-201945,B,BAE,5COP,SR,Domestic,Third,N/A - Perfect fit,2019,NaN,NaN,NaN,NaN,NaN,n/a - perfect fit


In [184]:
import string
string.punctuation
def remove_punctuations(text):
    punctuations = string.punctuation
    return text.translate(str.maketrans('','',punctuations))

In [185]:
df = df.fillna(" ")
df['Clean_Text'] = df['Clean_Text'].apply(lambda x : remove_punctuations(x))
df.head(2)

,Work Terms,College,Major,Concentration,Class BOT,Citizenship Status,Co-op #,suggestions,year,Terms,Coll,Conc 1,Citizenship,Coop #,Clean_Text
Responder ID,,,,,,,,,,,,,,,
12,201915-201925,B,FIN,5COP,JR,Domestic,Third,Please note that this is not directly related ...,2019,,,,,,please note that this is not directly related ...
27,201935-201945,B,BAE,5COP,SR,Domestic,Third,N/A - Perfect fit,2019,,,,,,na perfect fit


In [186]:
df['Clean_Text'].values[1]

'na  perfect fit'

In [187]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join(word for word in text.split() if word not in STOPWORDS)
df['Clean_Text'] = df['Clean_Text'].apply(lambda x : remove_stopwords(x))
df.head(2)

,Work Terms,College,Major,Concentration,Class BOT,Citizenship Status,Co-op #,suggestions,year,Terms,Coll,Conc 1,Citizenship,Coop #,Clean_Text
Responder ID,,,,,,,,,,,,,,,
12,201915-201925,B,FIN,5COP,JR,Domestic,Third,Please note that this is not directly related ...,2019,,,,,,please note directly related sachin interviewi...
27,201935-201945,B,BAE,5COP,SR,Domestic,Third,N/A - Perfect fit,2019,,,,,,na perfect fit


Removing Special Characters

In [188]:
from collections import Counter
word_count = Counter()
RARE_WORDS = set(word for(word,wc) in word_count.most_common()[:-10:-1])

def remove_rare_words(text):
    return " ".join([word for word in text.split() if word not in RARE_WORDS])

df['Clean_Text'] = df['Clean_Text'].apply(lambda x : remove_rare_words(x))
df.head(2)

,Work Terms,College,Major,Concentration,Class BOT,Citizenship Status,Co-op #,suggestions,year,Terms,Coll,Conc 1,Citizenship,Coop #,Clean_Text
Responder ID,,,,,,,,,,,,,,,
12,201915-201925,B,FIN,5COP,JR,Domestic,Third,Please note that this is not directly related ...,2019,,,,,,please note directly related sachin interviewi...
27,201935-201945,B,BAE,5COP,SR,Domestic,Third,N/A - Perfect fit,2019,,,,,,na perfect fit


In [189]:
def remove_spl_chars(text):
    text = re.sub('[^a-zA-Z0-9]'," ",text)
    text = re.sub('\s+',' ',text)
    return text

In [190]:
df['Clean_Text'] = df['Clean_Text'].apply(lambda x : remove_spl_chars(x))
df.head(2)

,Work Terms,College,Major,Concentration,Class BOT,Citizenship Status,Co-op #,suggestions,year,Terms,Coll,Conc 1,Citizenship,Coop #,Clean_Text
Responder ID,,,,,,,,,,,,,,,
12,201915-201925,B,FIN,5COP,JR,Domestic,Third,Please note that this is not directly related ...,2019,,,,,,please note directly related sachin interviewi...
27,201935-201945,B,BAE,5COP,SR,Domestic,Third,N/A - Perfect fit,2019,,,,,,na perfect fit


# Text Mining: Tokenization

In [191]:
from nltk.tokenize import word_tokenize
text_column = df['Clean_Text']
tokenized_text = text_column.apply(lambda x: word_tokenize(str(x)))
df.head(2)

,Work Terms,College,Major,Concentration,Class BOT,Citizenship Status,Co-op #,suggestions,year,Terms,Coll,Conc 1,Citizenship,Coop #,Clean_Text
Responder ID,,,,,,,,,,,,,,,
12,201915-201925,B,FIN,5COP,JR,Domestic,Third,Please note that this is not directly related ...,2019,,,,,,please note directly related sachin interviewi...
27,201935-201945,B,BAE,5COP,SR,Domestic,Third,N/A - Perfect fit,2019,,,,,,na perfect fit


In [192]:
for index, tokens in enumerate(tokenized_text[:5]): 
    print(f"Tokens in Row {index + 1}: {tokens}")

Tokens in Row 1: ['please', 'note', 'directly', 'related', 'sachin', 'interviewing', 'coop', 'experience', 'seems', 'lot', 'candidates', 'dont', 'fully', 'understand', 'banking', 'valuation', 'entirely', 'sure', 'curriculum', 'classes', 'centered', 'around', 'two', 'areas', 'would', 'highly', 'suggest', 'incorporating', 'basic', 'valuation', 'skills', 'go', 'long', 'way', 'field']
Tokens in Row 2: ['na', 'perfect', 'fit']
Tokens in Row 3: ['would', 'encourage', 'jalen', 'seek', 'courses', 'business', 'operations', 'organizational', 'leadership', 'would', 'also', 'encourage', 'look', 'basic', 'accounting', 'course', 'classes', 'help', 'budgeting']
Tokens in Row 4: ['na']
Tokens in Row 5: ['changes']


In [193]:
df['Tokenized_Column'] = tokenized_text

from nltk import bigrams as nltk_bigrams     
bigram_column = []
for tokens in tokenized_text:
    bi_grams = list(nltk_bigrams(tokens))
    bigram_column.append(bi_grams)

df['Bigrams'] = bigram_column
df.head(2)

,Work Terms,College,Major,Concentration,Class BOT,Citizenship Status,Co-op #,suggestions,year,Terms,Coll,Conc 1,Citizenship,Coop #,Clean_Text,Tokenized_Column,Bigrams
Responder ID,,,,,,,,,,,,,,,,,
12,201915-201925,B,FIN,5COP,JR,Domestic,Third,Please note that this is not directly related ...,2019,,,,,,please note directly related sachin interviewi...,"[please, note, directly, related, sachin, inte...","[(please, note), (note, directly), (directly, ..."
27,201935-201945,B,BAE,5COP,SR,Domestic,Third,N/A - Perfect fit,2019,,,,,,na perfect fit,"[na, perfect, fit]","[(na, perfect), (perfect, fit)]"


# Bigrams and frequencies

In [194]:
bigram_freq_dist = nltk.FreqDist()
for tokens in tokenized_text:
    bi_grams = list(nltk_bigrams(tokens))
    bigram_freq_dist.update(bi_grams)

top_n = 500
top_bigrams = bigram_freq_dist.most_common(top_n)


bigrams, frequencies = zip(*top_bigrams)

In [195]:
df_export = pd.DataFrame({
    'Bigram': [' '.join(bigram) for bigram in bigrams],
    'Frequency': frequencies,
})

df_export.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

merged_df = pd.concat([df, df_export], axis=1)
merged_df.head(2)

,Work Terms,College,Major,Concentration,Class BOT,Citizenship Status,Co-op #,suggestions,year,Terms,Coll,Conc 1,Citizenship,Coop #,Clean_Text,Tokenized_Column,Bigrams,Bigram,Frequency
0,201915-201925,B,FIN,5COP,JR,Domestic,Third,Please note that this is not directly related ...,2019,,,,,,please note directly related sachin interviewi...,"[please, note, directly, related, sachin, inte...","[(please, note), (note, directly), (directly, ...",academic preparation,44.0
1,201935-201945,B,BAE,5COP,SR,Domestic,Third,N/A - Perfect fit,2019,,,,,,na perfect fit,"[na, perfect, fit]","[(na, perfect), (perfect, fit)]",well prepared,35.0


Removing columns not required for visualization:

In [196]:
columns_to_drop = ['Coll', 'Conc 1', 'Coop #','Class BOT','Citizenship','Terms']
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df.head(2)

,Work Terms,College,Major,Concentration,Citizenship Status,Co-op #,suggestions,year,Clean_Text,Tokenized_Column,Bigrams,Bigram,Frequency
0,201915-201925,B,FIN,5COP,Domestic,Third,Please note that this is not directly related ...,2019,please note directly related sachin interviewi...,"[please, note, directly, related, sachin, inte...","[(please, note), (note, directly), (directly, ...",academic preparation,44.0
1,201935-201945,B,BAE,5COP,Domestic,Third,N/A - Perfect fit,2019,na perfect fit,"[na, perfect, fit]","[(na, perfect), (perfect, fit)]",well prepared,35.0


Exporting the merged dataframe:

In [197]:
merged_df.to_excel("bigram_frequency_tableau_ALL.xlsx", index=False)